# HOG file parsing for gene names

In [22]:
def convert_to_gene_id(name):
    if name.startswith('FRAX'):  # not FRAEX for excelsior
        return name[name.find('_')+1:name.rfind(' ')]
    if name.startswith('FRAEX'):
        return name[0:name.rfind(' ')]
    else:
        return name
assert convert_to_gene_id("FRAX01_FRAEX38873_V2_000052990.1_R0 [FRAX27_predicted_proteins_with_species_tag]") == "FRAEX38873_V2_000052990.1_R0"
tomato = 'Solyc03g095770.2.1 pacid=36135394 transcript=Solyc03g095770.2.1 locus=Solyc03g095770.2 ID=Solyc03g095770.2.1.ITAG2.4 annot-version=ITAG2.4 [Slycopersicum_390_ITAG2]'
assert convert_to_gene_id(tomato) == tomato


In [26]:
def parse_HOGs_from_fasta_directory():
    directory_HOGs = r"D:\josiah\Documents\Research\Thesis - Genome Symmetry\DNA_Duplications\data\HOGFasta"
    from glob import glob
    from os.path import join, splitext, basename
    file_list = glob(join(directory_HOGs, "*.fa"))
    HOGs = {}
    for filename in file_list:
        assert basename(filename).startswith('HOG')
        headers = []
        with open(filename, 'r') as fasta:
            for line in fasta:
                if line.startswith('>'):
                    headers.append(convert_to_gene_id(line[1:-1]))
        #assert len(headers) == len(set(headers)), "There was a redundant gene mention %s" % headers 
        HOGs[basename(splitext(filename)[0])] = headers
    return HOGs
HOGs = parse_HOGs_from_fasta_directory()
len(HOGs)

16539

# GFF parsing for overlapping annotations

TBD

# Greedy merge algorithm
 ( agglomerative clustering? )
* Starting clusters can be HOGs
* clusters = dict{ gene: pointer to cluster }
* Clusters are set(gene)
* First gene gets a new cluster
* If gene A overlaps with gene B anywhere, then go through each cluster to find gene A and add gene B
* If gene B is in any other clusters, find it and merge the entire cluster into gene A cluster
* Assert a gene can only ever be in one cluster at a time, except in the atomic operation of merging two


### Starting from HOGs but no side-effects, using networkx to retreive clusters

[Python Network Graphs](https://www.python.org/doc/essays/graphs/)

[Creating a networkx graph](https://networkx.github.io/documentation/networkx-1.10/tutorial/tutorial.html)  
[Graph Connected Components](https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.components.connected.connected_components.html)

In [27]:
def map_genes_to_HOGs(HOGs):
    genes_to_HOGs = {}
    for hog, genes in HOGs.items():
        for gene in genes:
            if gene not in genes_to_HOGs:
                genes_to_HOGs[gene] = set()
            genes_to_HOGs[gene].add(hog)
    return genes_to_HOGs

def test_map_genes_to_HOGs():
    genes_to_HOGs_answer = {'gene1': {'HOG1'},
                     'gene2': {'HOG1', 'HOG2'},
                     'gene3': {'HOG2'}}
    HOGs = {'HOG1': set(('gene1', 'gene2')),
            'HOG2': set(('gene2', 'gene3'))}  # all HOGs start out parsed and non-exlusive
    genes_to_HOGs = map_genes_to_HOGs(HOGs)
    print(genes_to_HOGs, '\n',genes_to_HOGs_answer)
    assert genes_to_HOGs == genes_to_HOGs_answer
test_map_genes_to_HOGs()    

{'gene1': {'HOG1'}, 'gene2': {'HOG2', 'HOG1'}, 'gene3': {'HOG2'}} 
 {'gene1': {'HOG1'}, 'gene2': {'HOG2', 'HOG1'}, 'gene3': {'HOG2'}}


In [28]:
from itertools import combinations
import networkx

def create_cluster_network(genes_to_HOGs, HOGs):
    g = networkx.Graph()
    g.add_nodes_from(HOGs.keys())
    for gene, hogs in genes_to_HOGs.items():
        if len(hogs) > 1:
            # two HOGs have overlap and need to be merged
            g.add_edges_from(combinations(hogs, 2))  # add edge between all hogs
            
    return g

In [29]:
def make_super_HOGs(HOGs, networked_hogs):
    """collect subnetworks together into larger clusters"""
    super_HOGs = {}
    genes_seen = set()
    clusters = [c for c in sorted(networkx.connected_components(networked_hogs), key=len, reverse=True)]
    for cluster in clusters:
        name = '-'.join(cluster)
        super_HOGs[name] = set().union(gene for hog in cluster for gene in HOGs[hog])
        assert super_HOGs[name] not in genes_seen, "You missed a clustering connection.  Genes should only occur once"
        genes_seen.update(super_HOGs[name])
            
    return super_HOGs

def test_make_super_HOGs():
    super_HOGs_answer = {'HOG1-HOG2': {'gene1','gene2','gene3'}}
    HOGs = {'HOG1': set(('gene1', 'gene2')),
            'HOG2': set(('gene2', 'gene3'))}  # all HOGs start out parsed and non-exlusive
    genes_to_HOGs = map_genes_to_HOGs(HOGs)
    network = create_cluster_network(genes_to_HOGs, HOGs)
    assert str(network.edges()) == "[('HOG1', 'HOG2')]", network.edges()
    super_HOGs = make_super_HOGs(HOGs, network)
    print(super_HOGs)
    assert super_HOGs == super_HOGs_answer, super_HOGs
test_make_super_HOGs()

{'HOG1-HOG2': {'gene1', 'gene2', 'gene3'}}


### Actual Clusters with Real HOGs

In [30]:
actual_genes_to_HOGs = map_genes_to_HOGs(HOGs)
actual_network = create_cluster_network(actual_genes_to_HOGs, HOGs)
super_HOGs = make_super_HOGs(HOGs, actual_network)

In [31]:
len(super_HOGs)

13645

In [32]:
largest_cc = max(networkx.connected_components(actual_network), key=len)

In [37]:
big_super = '-'.join(largest_cc)
big_super

'HOG27342-HOG25547-HOG9983-HOG27399-HOG2766-HOG27814-HOG2765-HOG2763'

In [38]:
super_HOGs[big_super]

{'FRAEX38873_V2_000009700.1_R0',
 'FRAEX38873_V2_000287630.1_R0',
 'FRAEX38873_V2_000292080.1_R0',
 'FRAEX38873_V2_000370050.1_R0',
 'FRAEX38873_v2_000009700.1 gene=FRAEX38873_v2_000009700',
 'FRAEX38873_v2_000370050.1 gene=FRAEX38873_v2_000370050',
 'OE6A037693P1 [OE6A]',
 'OE6A045665P1 [OE6A]',
 'OE6A100831P1 [OE6A]'}

In [41]:
for f in big_super.split('-'):
    print("cp %s.fa ../super_hog_test_viz/" % f)

cp HOG27342.fa ../super_hog_test_viz/
cp HOG25547.fa ../super_hog_test_viz/
cp HOG9983.fa ../super_hog_test_viz/
cp HOG27399.fa ../super_hog_test_viz/
cp HOG2766.fa ../super_hog_test_viz/
cp HOG27814.fa ../super_hog_test_viz/
cp HOG2765.fa ../super_hog_test_viz/
cp HOG2763.fa ../super_hog_test_viz/


In [51]:
len([c for c in sorted(networkx.connected_components(actual_network), key=len, reverse=True) if len(c) > 1])

2224

* output files with lists of gene names to look for in each HOG
* per species, find those gene names in the annotation
* count presence / absence of a gene name in an annotation
* group them back by super-HOGs
* End result: gene copy count per each gene family defined by a super-HOG

_Should I call this a multigene family?_